## 메타데이터 불러오기

- MovieLens 25M Dataset(https://grouplens.org/datasets/movielens/25m/) 기준
- 위 데이터에 없는 keyword, poster_path, 감독, 출연진 등 정보들 크롤링

In [ ]:
import pandas as pd
import json

metadata_df = pd.read_json('../movie_dataset_public_final/raw/metadata.json', lines=True)

## IMDB 크롤링

In [ ]:
!pip install imdbpy

     |████████████████████████████████| 298 kB 23.6 MB/s 


In [ ]:
from imdb import IMDb
import requests

In [ ]:
ia = IMDb()

id2metadata = {}

for idx, imdbId in tqdm(enumerate(metadata_df['imdbId']), total = len(metadata_df)):
    movie = ia.get_movie(imdbId)
    temp_metadata = {}
    for key in ['title', 'year', 'genre', 'runtimes', 'countries', 'languages', 'box office', 'kind', 'rating', 'votes', 'plot', 'synopsis', 'cover url', 'full-size cover url']:
        try: 
            temp_metadata[key] = movie[key]
        except:
            temp_metadata[key] = 'No'
    id2metadata[imdbId] = temp_metadata
    
    # if (idx+1) % 1000 == 0:
    #     with open('IMDB.json', 'w') as f:
    #         json.dump(id2metadata, f)
    #     pd.DataFrame(id2metadata).transpose().to_csv('IMDB.csv')

## TMDB 크롤링

In [ ]:
my_key = 'TMDB API key'

total_info = []
for idx, movie_id in tqdm(enumerate(tmdb_id), total=len(tmdb_id)):

    movie_url= f"https://api.themoviedb.org/3/movie/{movie_id}?api_key={my_key}"
    keyword_url = f"https://api.themoviedb.org/3/movie/{movie_id}/keywords?api_key={my_key}"
    credit_url = f"https://api.themoviedb.org/3/movie/{movie_id}/credits?api_key={my_key}"

    movie_response = requests.get(movie_url)
    keyword_response = requests.get(keyword_url)
    credit_response = requests.get(credit_url)

    movie_dict = movie_response.json()
    keyword_dict = keyword_response.json()
    credit_dict = credit_response.json()

    info = {**movie_dict, **keyword_dict, **credit_dict}

    # total_info.append(info)
    # if idx % 1000 == 0:
    #     pd.DataFrame(total_info).to_json('tmdb_crawling.json')
    #     pd.DataFrame(total_info).to_csv('tmdb_crawling.csv')


  0%|          | 0/62316 [00:00<?, ?it/s]

## MovieLens 크롤링
> Grouplens의 MovieLens Dataset보다 더 최근 영화들의 정보를 얻고자 Selenium을 통해 MovieLens(https://movielens.org) 크롤링

In [ ]:
# 필요패키지 import
import numpy as np
import pandas as pd
import time
import requests # 크롤링에 사용하는 패키지
from bs4 import BeautifulSoup # html 변환에 사용함
import json
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

driver = webdriver.Chrome('.//chromedriver.exe')
url = f'https://movielens.org/movies/{id_list[0]}'
driver.get(url)

cnt = 0
err_cnt = 0
for movieId in id_list:
    url = f'https://movielens.org/movies/{movieId}'
    driver.get(url)
    cnt += 1
    tt = 1 + np.random.rand(1)[0]        
    time.sleep(tt)
    try:
        text_list.append({'id':movieId, 'raw':raw})
        title = driver.find_element_by_xpath('//*[@id="main-container"]/ui-view/movie-page/div/div[1]/div[2]/h1').text
        genre = driver.find_element_by_xpath('//*[@id="main-container"]/ui-view/movie-page/div/div[1]/div[2]/div[3]/div[2]/div[1]').text
        #genre = genre.split('\n')[1:]
        overview = driver.find_element_by_xpath('//*[@id="main-container"]/ui-view/movie-page/div/div[2]/div[1]/p').text
        
        director = driver.find_element_by_xpath('//*[@id="main-container"]/ui-view/movie-page/div/div[2]/div[2]/div[2]/span/a').text
        cast = driver.find_element_by_xpath('//*[@id="main-container"]/ui-view/movie-page/div/div[2]/div[2]/div[3]').text
        #cast = cast.split('\n')[1].split(',')[:-1]
        movie_dict = {'id':movieId, 'title':title, 'genre':genre, 'overview':overview, 'director':director, 'cast':cast}
        datas.append(movie_dict)
    except:
        movie_dict = {'id':movieId, 'title':0, 'genre':0, 'overview':0, 'director':0, 'cast':0}
        datas.append(movie_dict)
        err_cnt += 1
        print(f'{cnt}번째 영화에서 {err_cnt}번째 error')
        
    if cnt % 300 == 0:
        df = pd.DataFrame(datas)
        df.to_csv(f'dt0_{cnt}.csv', index=False)
        datas = []
df = pd.DataFrame(datas)
df.to_csv(f'dt0_{cnt}.csv', index=False)